#### SQL Prompts to create 'master' table for EDA. Contains all hospital events, EKGs, and labels

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.colab import userdata
projectid=userdata.get('project_ID')

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import os
import db_dtypes

In [ ]:
#filtering data to adults with non-missing age
%%bigquery --project={projectid}
CREATE OR REPLACE TABLE `{projectid}.ekg_labels.icd_labels` AS
SELECT *
FROM `ekg-classification.ekg_labels.records_w_diag_icd10`
WHERE age > 17 AND age IS NOT NULL
ORDER BY study_id ASC, subject_id ASC, ecg_time ASC;

Query is running:   0%|          |

""


In [ ]:
#Creates dataset with all possible hospital events + EKG data. Induces duplicates for 2 reasons: joining ED + hosp events by a coalesced hadm_id, then joining EKGs during admissions to that coalesced ID.
%%bigquery --project={projectid}
CREATE OR REPLACE TABLE `{projectid}.ekg_data.all_events` AS
WITH ed_hosp as (SELECT
    COALESCE(a.subject_id, b.subject_id) AS subject_id1,
    COALESCE(a.hadm_id,   b.hadm_id)    AS hadm_id1,
    a.*,
    b.* EXCEPT(subject_id, hadm_id, race)
  FROM `physionet-data.mimiciv_ed.edstays` AS a
  FULL OUTER JOIN `physionet-data.mimiciv_3_1_hosp.admissions` AS b
    ON (a.subject_id = b.subject_id
   AND a.hadm_id = b.hadm_id)
   ORDER BY subject_id1, hadm_id1
)

SELECT
  v.subject_id1,
  v.hadm_id1,
  v.* EXCEPT(subject_id1, hadm_id1),
  c.* EXCEPT(subject_id, hadm_id, stay_id, intime, outtime),
  d.* EXCEPT(subject_id, file_name, study_id, gender),
  e.* EXCEPT(subject_id, file_name, study_id, ecg_time),
  c.stay_id as icu_stayid,
  d.file_name AS ecg_file,
  e.file_name AS label_file,
  d.study_id AS ecg_id,
  e.study_id AS label_id
FROM ed_hosp AS v
LEFT JOIN `physionet-data.mimiciv_3_1_icu.icustays` AS c
  ON (v.subject_id1 = c.subject_id
  AND v.hadm_id1 = c.hadm_id)
LEFT JOIN `{projectid}.ekg_labels.icd_labels` AS d
  ON (v.subject_id1 = d.subject_id AND (
            (v.hadm_id1 IS NOT NULL AND d.hosp_hadm_id = v.hadm_id1)
         OR (d.hosp_hadm_id IS NULL)))
LEFT JOIN `physionet-data.mimiciv_ecg.record_list` AS e
  ON d.study_id = e.study_id;

Query is running:   0%|          |

""


In [ ]:
#Check missingness
%%bigquery --project={projectid}
SELECT COUNT(*) AS missing_ids
FROM `{projectid}.ekg_data.all_events`
WHERE subject_id1 IS NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,missing_ids
0,0


#### *Updated join fixed missing subject_Ids when subject was in either ED OR HOSP table, but not both*

###### **Note: these prompts will nopt run with {projectid} in the sql line creating/replacing a table because SQL does not recognize {}. This is intentionally hidden for pushing this code to Github.**

### Updates: After some thought about what I want the model to do and the data that would be appropriate for that problem, I realize there is a much simpler way to join the data. Table A: all hospital events (joins edstays, admissions, and icustays). Table B: joins EKG diagnostic data to the record itself. I will use a distinct list of subject_id's in table B to limit records in table A then, after summarizing my variables of interest, join the two tables.

In [19]:
%%bigquery --project={projectid}
CREATE OR REPLACE TABLE `{projectid}.ekg_data.all_events` AS
WITH ed_hosp as (SELECT DISTINCT
    COALESCE(a.subject_id, b.subject_id) AS subject_id1,
    a.*,
    b.* EXCEPT(subject_id, hadm_id, race),
    b.hadm_id as hosp_hadm_id
  FROM `physionet-data.mimiciv_ed.edstays` AS a
  FULL OUTER JOIN `physionet-data.mimiciv_3_1_hosp.admissions` AS b
  ON (a.subject_id = b.subject_id AND (
            (a.hadm_id IS NOT NULL AND a.hadm_id = b.hadm_id)
         OR (b.hadm_id IS NULL)))
  ORDER BY subject_id1, hadm_id
)

SELECT
  v.subject_id1,
  v.* EXCEPT(subject_id1),
  c.* EXCEPT(subject_id, hadm_id, stay_id, intime, outtime)
FROM ed_hosp AS v
LEFT JOIN `physionet-data.mimiciv_3_1_icu.icustays` AS c
  ON (v.subject_id1 = c.subject_id
  AND v.hosp_hadm_id = c.hadm_id)
ORDER BY subject_id1, hadm_id, stay_id, admittime;

Query is running:   0%|          |

""


In [22]:
%%bigquery --project={projectid}
CREATE OR REPLACE TABLE `{projectid}.ekg_data.all_ekg` AS
SELECT DISTINCT
  a.*,
  b.* EXCEPT(subject_id, file_name, study_id, ecg_time),
  b.file_name as ecg_file,
  b.study_id as ecg_id,
  b.ecg_time as ecg_time1
  FROM `{projectid}.ekg_labels.icd_labels` as a
  LEFT JOIN `physionet-data.mimiciv_ecg.record_list` as b
  ON a.subject_id=b.subject_id AND a.study_id=b.study_id
  ORDER BY a.subject_id, a.study_id, a.ecg_time;

Query is running:   0%|          |

""


In [23]:
%%bigquery --project={projectid}
CREATE OR REPLACE TABLE `{projectid}.ekg_data.unique_subjectid` AS
SELECT DISTINCT subject_id from `ekg-classification.ekg_data.all_ekg`;

Query is running:   0%|          |

""
